In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("Sentiment_train.tsv",delimiter='\t',quoting=3)

### 0 - negative,1 - somewhat negative,2 - neutral,3 - somewhat positive,4 - positive

In [3]:
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [4]:
import tensorflow as tf

In [5]:
pd.crosstab(df['Sentiment'],1)

col_0,1
Sentiment,
0,7072
1,27273
2,79582
3,32927
4,9206


In [6]:
from sklearn.cross_validation import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df['Phrase'],df['Sentiment'],test_size=0.20,random_state=42)

/usr/lib64/python3.4/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(124848,)
(31212,)
(124848,)
(31212,)


In [8]:
from tensorflow.python.keras.preprocessing.text import Tokenizer

In [9]:
topnum=5000

In [10]:
t=Tokenizer(num_words=topnum)

In [11]:
t.num_words

5000

In [12]:
t.fit_on_texts(x_train.tolist())

In [13]:
x_train=t.texts_to_sequences(x_train.tolist())

In [14]:
x_test=t.texts_to_sequences(x_test.tolist())

In [15]:
x_test

[[6, 11, 3798],
 [221, 60, 4, 379, 118, 258, 709, 19, 96, 416, 41, 5, 20, 2266, 199],
 [3, 173, 532],
 [34, 14, 690],
 [37, 4620],
 [63, 46, 326, 4, 2229, 5, 14, 816, 126],
 [714, 133, 545, 57, 342, 840],
 [2402, 5, 20, 1530],
 [],
 [1790, 718, 143],
 [1, 940, 8, 2391, 4, 1, 2019, 470],
 [],
 [2218,
  1,
  586,
  9,
  1,
  87,
  15,
  8,
  2,
  1286,
  1944,
  290,
  4,
  9,
  2927,
  6,
  920,
  1478,
  37,
  1745,
  7,
  10],
 [],
 [448, 3, 563],
 [6, 2],
 [3327],
 [2, 2706, 4020, 136, 4],
 [1586],
 [1741, 31],
 [88, 5, 259, 95],
 [2, 25, 1079, 409, 7, 1, 293],
 [16, 4030, 454],
 [5, 541, 2, 1788, 3549],
 [3138, 26, 126],
 [],
 [54, 4],
 [12, 18, 427, 3, 236, 5, 85, 104, 284, 1121],
 [574, 2147],
 [8, 7, 1, 120],
 [1894, 6, 712, 184, 5, 2243, 4, 1, 557],
 [81, 1603, 26, 286, 4, 3125, 6, 3935, 148, 2668],
 [],
 [3, 1054, 4, 3987],
 [3508, 1513, 15],
 [375, 105],
 [719, 1667, 6],
 [16, 1817, 298, 15, 5, 305],
 [160, 36, 73, 2449, 21, 1, 4129, 3, 40, 331],
 [2,
  219,
  3,
  151,
  13,


In [16]:
len(x_train[1])

2

In [17]:
max_len=80

In [18]:
from tensorflow.python.keras.preprocessing import sequence


In [19]:
x_train=sequence.pad_sequences(x_train,maxlen=max_len,padding='post')


In [20]:
x_test=sequence.pad_sequences(x_test,maxlen=max_len,padding='post')

In [21]:
x_test

array([[   6,   11, 3798, ...,    0,    0,    0],
       [ 221,   60,    4, ...,    0,    0,    0],
       [   3,  173,  532, ...,    0,    0,    0],
       ...,
       [   1,  750, 4125, ...,    0,    0,    0],
       [2140,  386,    0, ...,    0,    0,    0],
       [   1,  469, 3132, ...,    0,    0,    0]], dtype=int32)

In [22]:
print(x_train.shape)

(124848, 80)


In [23]:
print(x_test.shape)

(31212, 80)


In [24]:
embedding_vector_length = 30

In [25]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Embedding,Flatten,Dropout,Dense

In [26]:
model=Sequential()

In [27]:
model.add(Embedding(input_dim=topnum+1,output_dim=embedding_vector_length,input_length=max_len))

In [28]:
model.add(Flatten())

In [29]:
model.add(Dense(300,activation='relu',input_shape=(80,)))

In [30]:
model.add(Dense(200,activation='relu'))

In [31]:
model.add(Dropout(0.3))

In [32]:
model.add(Dense(100,activation='relu'))

In [33]:
model.add(Dense(60,activation='relu'))

In [34]:
model.add(Dense(5,activation="softmax"))

In [35]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [36]:
#pd.crosstab(y_train,1)
from tensorflow.python.keras.utils import to_categorical
y_train = to_categorical(y_train,num_classes=5)

model.fit(x_train,y_train,# validation_data=(x_test,y_test),
          epochs=10,batch_size=10,shuffle=True)